## Example of displaying heatmap using folium

### Import libraries  

In [ ]:
import pandas as pd
import folium

### Parameters

In [ ]:
csv_in = 'pop_move_H27_H28-utf8.csv'

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Read CSV file

In [ ]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())
#display(df)

### Calculation of increase/decrease of population  

In [ ]:
df['増減'] = df['転入'] - df['転出']
display(df.head())

### Drop lines other than Prefs  

In [ ]:
df['都道府県'].str.match(r'^(.*市|.*計|.*区部)$')  # debug

In [ ]:
df2 = df.drop(df.index[df['都道府県'].str.match(r'^(.*市|.*計|.*区部)$') ]).reset_index(drop=True)
print(df2.info())
display(df2.head())

### Check data  

In [ ]:
print(df2.loc[df2['増減'].idxmin()])
print(df2.loc[df2['増減'].idxmax()])

### Parameters2 (for lat/lon)   

In [ ]:
csv_in2 = 'h3010pref_pos_utf8.csv'

### Read CSV file

In [ ]:
df_pos = pd.read_csv(csv_in2, delimiter=',', skiprows=0, header=0)
print(df_pos.shape)
display(df_pos.head())

### Merge DataFrames to add lat/lon to pref data  

In [ ]:
df_merged = pd.merge(df2, df_pos, left_on='都道府県', right_on='name', how='left')
print(df_merged.shape)
display(df_merged.head())

### Make data for heatmap  
[ [lat2, lon2, value2], [lat2, lon2, value2], ... ]  

In [ ]:
df_heat = df_merged[['lat', 'lon', '増減']]
print(df_heat.shape)
display(df_heat.head())

### Make map  

In [ ]:
from folium.plugins import HeatMap

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12))
display(m)

### Add color bar  

In [ ]:
from folium.plugins import HeatMap
import branca.colormap as cm

m = folium.Map(location=[35, 140], zoom_start=4)
m.add_child(HeatMap(df_heat, radius=12,
                    gradient={0:'blue', 0.5:'lime', 1:'red'}))

min = df_heat['増減'].min()
max = df_heat['増減'].max()
colors = ['blue', 'lime', 'red']
colormap = cm.LinearColormap(colors, vmin=min, vmax=max).to_step(10000)
colormap.caption = 'Increase/Decrease of Population'
m.add_child(colormap)

display(m)